In [ ]:
# Script to cluster GAEZ data using Agglomerative Heirarichal clustering method

import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import os
import sys
import seaborn as sns; sns.set(color_codes=True)
import matplotlib
import matplotlib.style as style
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import itertools
import scipy.cluster.hierarchy as hac

from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster

from tkinter import filedialog
from tkinter import *

%matplotlib auto

print('Python version ' + sys.version)
print('Matplotlib version ' + matplotlib.__version__)
print('Pandas version ' + pd.__version__)


In [ ]:
# Prompt user to select folder that contains required data files
root = Tk()
#root.folder =  filedialog.askdirectory()
#root.destroy()

root.folder = os.getcwd()

In [ ]:
# Set user-defined parameters 


# Vietnam

regions = {1: "NOE",
           2: "NOW",
           3: "RRD",
           4: "NOC",
           5: "SOC",
           6: "SOE",
           7: "MRD"} # Regions to be clustered


'''
regions = {1: "VNM"
          }

'''
'''
# Viet Nam
k = {1: 7,
     2: 6,
     3: 9,
     4: 15,
     5: 13,
     6: 9,
     7: 9,
    } # Selected number of clusters, using 0.2 threshold in elbow graphs
'''


## Viet Nam
k = {1: 3,
     2: 2,
     3: 2,
     4: 2,
     5: 3,
     6: 2,
     7: 2,
    } # Selected number of clusters, using 0.2 threshold in elbow graphs


'''
k = {1:15
    }
'''

## Crop list for Viet Nam
crop_list = {'Cassava': ['Cassava'],
             'Coffee': ['Coffee'],
             'Maize': ['Maize'],
             'Other': ['Sweet potato'],
             'Rice': ['Wetland rice'],
             'Sugarcane': ['Sugarcane'],
             #'Sorghum': ['Sorghum'],
             #'Groundnuts': ['Groundnut'],
             #'Cocoa': ['Cocoa'],
             #'Beans': ['Phaseolus bean'],
             #'Rubber': ['Jatropha'],
             'Vegetables and fruits': ['Cabbage','Carrot','Onion','Tomato','Banana','Citrus','Coconut'],
             #'Sweet potato': ['Sweet potato']
            }

#land_area_cell = 0.08554 # in 1000 sq.km (1000 sq.km = 100 kha)
#land_area_cell = 0.0779868 # for Indonesia in 1000 sq.km 
land_area_cell = 0.07793736755 # for Viet Nam in 1000 sq.km 
#land_area_cell = 0.079871871 # for Nicaragua and Honduras in 1000 sq.km
#land_area_cell = 0.08333 # for Nicaragua and Honduras in 1000 sq.km

input_data_location_yld = os.path.join(root.folder, r'data\vietnam_attainable_yield_all_crops.csv')
input_data_location_lnd = os.path.join(root.folder, r'data\vietnam_land_cover_types.csv')
input_data_location_reg = os.path.join(root.folder, r'data\vietnam_regions_v3.csv') # Location of file with regional division of GAEZ data

### Water clustering
water_included = True

if water_included:
    input_data_location_cwd = os.path.join(root.folder, r'data\vietnam_crop_water_deficit_all_crops.csv') # Location of file crop water deficit data
    input_data_location_evt = os.path.join(root.folder, r'data\vietnam_evapotranspiration_all_crops.csv') # Location of file evapotranspiration data
    input_data_location_prc = os.path.join(root.folder, r'data\vietnam_precipitation.csv') # Location of file precipitation data

write_to_csv = False # Set as 'True' or 'False' to write an output file or not  

plot_elbow = False # Set as 'True' or 'False' to plot 'Elbow graph' or not. The 'Elbow graph' helps select an appropriate number of clusters for a given dataset   
plot_clusters = False # Set as 'True' or 'False' to plot results by cluster or not.
plot_map = True # Set as 'True' or 'False' to plot map of cells by cluster or not.
plot_cropstats = False # Set as 'True' or 'False' to plot crop yield statistics by crop and region

In [ ]:
df = pd.read_csv(input_data_location_yld, header=None) # Read data for crop yields
df_lc = pd.read_csv(input_data_location_lnd, header=None) # Read data for land area by land cover type
df_regions = pd.read_csv(input_data_location_reg, header=None) # Read data file for regions

if water_included: 
    df_cwd = pd.read_csv(input_data_location_cwd, header=None) # Read data for crop water deficit
    df_evt = pd.read_csv(input_data_location_evt, header=None) # Read data for evapotranspiration
    df_prc = pd.read_csv(input_data_location_prc, header=None) # Read data for precipitation

def read_prep_df(df_name):
    column_names = ['column id', 'row number'] + list(range(0,len(df_name.columns)-2))
    df_name.columns = column_names
    df_name = df_name.melt(['column id','row number'], var_name='col number', value_name='region' if df_name['column id'].any() == 'region' else 'value') # Reformat table from wide to long
    df_name = df_name.pivot_table(index=['row number','col number'], columns='column id', values='region' if df_name['column id'].any() == 'region' else 'value') #Set crop combinations as columns
    return df_name

In [ ]:
# OPTIONAL: Plot values for each cluster

def print_clusters(df, Z, k, plot=False):
    # k Number of clusters I'd like to extract
    results = fcluster(Z, k, criterion='maxclust')

    # check the results
    s = pd.Series(results)
    clusters = s.unique()
    x = df_plot_2.columns
    x_ticks = pd.Series(x)    
    
    for c in clusters:
        cluster_indices = s[s==c].index
        #print("Cluster %d contains %d clusters" % (c, len(cluster_indices)))
        #print(cluster_indices)
        if plot:
            ax = df_plot_2.T.iloc[:,cluster_indices].plot(legend=None)
            ax.set_ylabel('Crop yield (t/ha)')
            ax.set_xlabel('Crop combination')
            plt.xticks(np.arange(len(x)), x, rotation='vertical')
            plt.tight_layout()
            plt.savefig(root.folder + 
                        r'\output\figs\clusters_' + 
                        regions[region] + 
                        '_cluster' + 
                        str(c)  + 
                        '.pdf', 
                        bbox_inches='tight', 
                        pad_inches=0)
            #plt.show()
            
def crop_selection(df_all):
    for each in df_all.columns:
        if ' '.join(each.split(' ')[:-2]) not in list(itertools.chain.from_iterable(crop_list.values())):
            df_all.drop(each, axis=1, inplace=True)
    return df_all

def filter_df(df_all):
    df_new = df_all.loc[df_all['region'] == region].drop('region', axis=1)
    df_new = df_new.replace(-9, np.nan)
    df_new = df_new.dropna(thresh=1)
    df_new[df_new<0]=0
    return df_new

def df_renumber_index(df,kmax):
    df.loc[~df['index'].isin(list(range(1,kmax+1))), 'index'] = 0
    #df = df.set_index('index')
    return df

def map_plot(df):
    plt.figure()
    cluster_df = pd.DataFrame(df_plot_1['cluster'].value_counts()).sort_index().reset_index()
    cluster_df['label names'] = cluster_df['index'].astype(str) + ' (' + cluster_df['cluster'].astype(str) + ' cells)'
    labels = pd.Series(cluster_df['index'])
    label_names = pd.Series(cluster_df['label names'])
    cluster_map = df_plot_1.reset_index().pivot("row number", "col number", "cluster")
    
    # add a cbar_kws={"boundaries": linspace(-1, 1, 4)} to the heatmap invocation
    # to have it generate a discrete colorbar instead of a continous one.
    
    n = len(label_names)
    cmap = sns.color_palette("hls", n)

    sns.set(style = 'white',
            font = 'Arial',
            context = 'paper')
    ax = sns.heatmap(cluster_map, cmap=cmap)
    ax.set_aspect('equal')
    
    colorbar = ax.collections[0].colorbar
    r = colorbar.vmax - colorbar.vmin
    colorbar.set_ticks([colorbar.vmin + 0.5 * r / (n) + r * i / (n) for i in range(n)])
    #colorbar.set_ticks(labels)
    colorbar.set_ticklabels(label_names)

    ax.set_xlabel('')
    ax.set_ylabel('')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('Map of clusters for region ' + str(regions[region]))
    
    plt.tight_layout()
    
    return plt.savefig(root.folder + 
                       r'\output\figs\clustermap_' + 
                       regions[region] + 
                       '.svg', 
                       bbox_inches='tight', 
                       pad_inches = 0)
    #plt.show()    

In [ ]:
# Join region dataframe to crop yields and land cover data frames Region column
df = read_prep_df(df)
crop_selection(df)            

combine_list = {}
for each_crop in crop_list.keys():
        for each_column in df.columns:
            if ' '.join(each_column.split(' ')[:-2]) in crop_list[each_crop]:
                merge_column_name = str(each_crop) + ' ' + ' '.join(each_column.split(' ')[-2:])
                combine_list[merge_column_name] = [(str(sub_crop) + ' ' + ' '.join(each_column.split(' ')[-2:])) for sub_crop in crop_list[each_crop]]

df_lc = read_prep_df(df_lc)
df_lc = df_lc.mul(land_area_cell/100) # Calculate area of different land cover type for each cell in 1000 sq.km
df_regions = read_prep_df(df_regions)
df_regions = df_regions.astype(int)

    
df = df.join(df_regions)
df_lc = df_lc.join(df_regions)

if water_included:
    df_cwd = read_prep_df(df_cwd)
    crop_selection(df_cwd)
    df_cwd = df_cwd.join(df_regions)
    
    df_evt = read_prep_df(df_evt)
    crop_selection(df_evt) 
    df_evt = df_evt.join(df_regions)
    
    df_prc = read_prep_df(df_prc)
    df_prc = df_prc.join(df_regions)


# Filter dataframe to keep only selected region
region_loop = 0

for region in regions.keys():
    
    df_selected = filter_df(df)
    #df_selected = df.loc[df['region'] == region].drop('region', axis=1)
    df_lc_selected = df_lc.loc[df_lc['region'] == region].drop('region', axis=1)
    
    if water_included:
        #df_cwd_selected = df_cwd.loc[df_cwd['region'] == region].drop('region', axis=1)
        #df_evt_selected = df_evt.loc[df_evt['region'] == region].drop('region', axis=1)
        #df_prc_selected = df_prc.loc[df_prc['region'] == region].drop('region', axis=1)
        df_cwd_selected = filter_df(df_cwd)
        df_evt_selected = filter_df(df_evt)
        df_prc_selected = filter_df(df_prc)
        
    for each in combine_list:
        if each not in df_selected.columns:
            df_selected[each] = df_selected[combine_list[each]].mean(axis=1)
            df_selected.drop(combine_list[each], axis=1, inplace=True)
            if water_included:
                df_cwd_selected[each] = df_cwd_selected[combine_list[each]].mean(axis=1)
                df_cwd_selected.drop(combine_list[each], axis=1, inplace=True)
                df_evt_selected[each] = df_evt_selected[combine_list[each]].mean(axis=1)
                df_evt_selected.drop(combine_list[each], axis=1, inplace=True)

    df_selected = df_selected.reindex(sorted(df_selected.columns), axis=1)
    crop_order = list(df_selected.columns)
    
#########################################
    df_selected_by_crop = df_selected

#########################################
        
    # Data normalization using min-max (0-1)
    normalized_df = (df_selected-df_selected.min())/(df_selected.max()-df_selected.min())
    normalized_df.fillna(0, inplace=True)
    
    # Agglomerative Heirarchical clustering
    Z = hac.linkage(normalized_df, 
                    method='ward', 
                    metric='euclidean') # Select type of Agglomerative Heirarchical clustering
    results = fcluster(Z, k[region], criterion='maxclust') # Select cut-off. k = number of clusters
    s = pd.Series(results)
    
    # Save df_5 to plot clusters
    df_plot_2 = normalized_df
    
    # Plot map of cells by cluster
    df_selected['cluster'] = s.values
    df_plot_1 = df_selected[['cluster']]
    
    if plot_map == True:
        map_plot(df_plot_1)
    
    df_plot_1['region'] = str(regions[region])
    df_plot_1.reset_index(inplace=True)
    if region_loop == 0:
        df_clustermap_all = pd.DataFrame(columns=df_plot_1.columns)
    df_clustermap_all = df_clustermap_all.append(df_plot_1)
    if plot_cropstats == False:
        region_loop += 1  
       
    print_clusters(df_selected, Z, k[region], plot=plot_clusters)
    
    # Plot 'Elbow' graph
    last = Z[-25:, 2]
    last_rev = last[::-1]
    error_ratio = (last_rev/max(last_rev)).round(2)
    idxs = np.arange(1, len(last) + 1)
    
    if plot_elbow==True:
        plt.figure()
        plt.plot(idxs, error_ratio, marker = 'o', markersize = 5)
        plt.xlabel('Number of clusters')
        plt.ylabel('% of max. error')
        plt.title('Elbow graph for region ' + str(regions[region]))
        
        for i,j in zip(idxs, error_ratio):
            plt.annotate(str(j), xy=(i,j), ha='left', fontsize = 12)
        plt.savefig(root.folder + 
                    r'\output\figs\elbow_graph_' + 
                    regions[region] + 
                    '.pdf',  
                    bbox_inches='tight', 
                    pad_inches = 0)
        #plt.show()
    
    # Aggregate data into clusters. Sum the number of cells in each cluster and calculate mean scores for each crop combination
    for each in df_selected.columns:
        if each != 'cluster':
            df_selected[each] = df_selected[each].div(10)    
    df_selected = df_selected.join(df_lc_selected)
    df_selected['count'] = 1
    aggregator = {}
    
    for each in df_selected.columns:
        if each not in ['cluster','count'] + list(df_lc.columns) :
            aggregator[each] = 'mean'
        if each in list(df_lc.columns) :
            aggregator[each] = 'sum'
    aggregator['count'] = 'sum'
    
    df_6 = df_selected.groupby(['cluster']).agg(aggregator).astype(float) # Sum the number of cells in each cluster and calculate mean scores for each crop combination
    df_6['land_area_total'] = df_6['count']*land_area_cell # Calculate land area of each cluster
    #df_7 = df_6.drop(['count'], axis=1)
    
    # Reorder columns
    col_order = ['count','land_area_total'] + list(df_lc_selected.columns) + crop_order
    df_6 = df_6[col_order]
    
################################    
    if plot_cropstats:
    
        df_preclustering_by_crop = {}
        
        df_preclustering = df_selected_by_crop.append(df_6).fillna(0).mul(10)
        filter_col = [col for col in df_6.columns if col.startswith(tuple(crop_list.keys()))]
        df_preclustering = df_preclustering[filter_col].reset_index()
        df_preclustering = df_renumber_index(df_preclustering,k[region])
        
        #df_preclustering.to_csv(os.path.join(root.folder, 'output\preclustering_results' + str(regions[region]) +'.csv'))
        
        df_preclustering_melt = pd.melt(df_preclustering, id_vars='index', 
                                        value_vars=[x for x in df_preclustering.columns if x != 'index'])
        df_preclustering_melt['crop'] = [' '.join(x.split(' ')[:-2]) for x in df_preclustering_melt['variable']]
        df_preclustering_melt['variable'] = [' '.join(x.split(' ')[-2:]) for x in df_preclustering_melt['variable']]
        df_preclustering_melt['region'] = region
        
        if region_loop == 0:
            df_preclustering_all = pd.DataFrame(columns=df_preclustering_melt.columns)
            
        df_preclustering_all = df_preclustering_all.append(df_preclustering_melt)
        region_loop += 1   
        
            
################################

    if water_included:
        df_cluster = pd.DataFrame(df_selected['cluster'])
        
        df_cwd_selected = df_cluster.join(df_cwd_selected)
        df_cwd_selected = df_cwd_selected.groupby(['cluster']).agg('mean').astype(float).div(1000)
        
        df_evt_selected = df_cluster.join(df_evt_selected)
        df_evt_selected = df_evt_selected.groupby(['cluster']).agg('mean').astype(float).div(1000)
        
        df_prc_selected = df_cluster.join(df_prc_selected)
        df_prc_selected = df_prc_selected.groupby(['cluster']).agg('mean').astype(float).div(1000)

        
    # Print results in csv   
    if write_to_csv == True:
        df_6.to_csv(os.path.join(root.folder, r'output\clustering_results_' + str(regions[region]) +'.csv'))
        
        if water_included:
            df_cwd_selected.to_csv(os.path.join(root.folder, r'output\clustering_results_cwd_' + str(regions[region]) +'.csv'))
            df_evt_selected.to_csv(os.path.join(root.folder, r'output\clustering_results_evt_' + str(regions[region]) +'.csv'))
            df_prc_selected.to_csv(os.path.join(root.folder, r'output\clustering_results_prc_' + str(regions[region]) +'.csv'))
     

In [ ]:
if plot_cropstats:
    #df_preclustering_all['index'][df_preclustering_all['index']!=0] = 1
    df_preclustering_all['index'][df_preclustering_all['index']==0] = 'actual'
    df_preclustering_all['index'][df_preclustering_all['index']!='actual'] = 'cluster'
    
    for each_crop in crop_list.keys():
        '''plt.figure(figsize=(30, 30), 
                   dpi= 80, 
                   facecolor='w', 
                   edgecolor='k')'''
        #sns.set_style('white')
        sns.set(style = 'white',
                font = 'Arial',
                context = 'talk')
        #sns.set(font_scale=1.4)

        '''if len(regions.keys()) > 1:
            g = sns.FacetGrid(df_preclustering_all[df_preclustering_all['crop']==each_crop], 
                              col='region', 
                              col_wrap=2, 
                              palette="Set1",)     
            g = (g.map(sns.violinplot, 'variable', 'value', 'index').
                 add_legend().
                 set_axis_labels('','Yield (t/ha)'))
            axes = g.axes.flatten()
            g.fig.suptitle(each_crop)   
            for each_region in regions.keys():  
                axes[each_region-1].set_title(str(regions[each_region]))
        '''
        #else:
        g = sns.catplot(x="variable", 
                        y="value",
                        hue="index", 
                        col='region',
                        col_wrap=1 if len(regions)==1 else 2,
                        cut=0,
                        data=df_preclustering_all[df_preclustering_all['crop']==each_crop], 
                        kind="violin", 
                        #split=True,
                        ci=None,
                        palette='muted',
                        linewidth=2.5,
                        #height=4, 
                        #aspect=1
                       )
        g._legend.set_title('')
        plot_titles = '' if len(regions)==1 else '{col_name} {col_var}'  
        (g.set_axis_labels('','Yield (t/ha)')
         .set_titles(plot_titles)
         .despine(left=True))
        g.fig.suptitle(each_crop)   
        
        #plt.xticks(rotation=90)
        for ax in g.axes.flat:
            for label in ax.get_xticklabels():
                label.set_rotation(90)
        #plt.gca().set_ylim(bottom=0)
        #plt.show()
        plt.savefig(root.folder + 
                    r'\output\figs\crop_stats_' + 
                    each_crop + 
                    '.svg',
                    bbox_inches='tight', 
                    pad_inches = 0,
                    dpi=300
                   )
    

In [ ]:
# Print table of all row numbers, col numbers, clusters, and regions
df_clustermap_all['cluster'] = df_clustermap_all['cluster'].map('{:02}'.format)
df_clustermap_all['cluster_unique'] = df_clustermap_all['region'] + 'C' + df_clustermap_all['cluster'].astype(str)
df_clustermap_all.drop(['cluster', 'region'], 
                       axis=1, 
                       inplace=True)

#df_clustermap_all['count'] = df_clustermap_all.groupby(['cluster_unique'])['cluster_unique'].transform('count')
#df_clustermap_all['cluster_land_area'] = df_clustermap_all['count'].mul(land_area_cell)
df_clustermap_all.to_csv(os.path.join(root.folder, r'output\clustermap_table.csv'), index=None)